# Regional Work


## Selecting Sub Regions

It is quite common to be interested in a sub-region.

Let's for example take a look at *El Nino 3-4* region defined here as the area comprised between longitudes  170W and 120W and between latitudes 5S and 5N.

Let's compute a time serie for this area

In [1]:
# Load data
import cdms2
ipsl_tas_file = cdms2.open("/global/cscratch1/sd/cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/tas/gr/v20180803/tas_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc")
tas = ipsl_tas_file('tas', time=("2000", "2010", "con"), latitude=(-5,5,'ccb'), longitude=(-170, -120,'con')) # 10 years over ninon 34

# Compute Spatial Average
import genutil
el_nino34 = genutil.averager(tas, axis='xy')
# Print the first year
print(el_nino34[:12])

[299.22178277778255 299.4175441483776 299.9929696423216 300.58908940992745
 300.62121497727253 299.7694262325941 299.32684459793325 298.94147435427
 298.49743389123864 298.13526489092146 297.7076472710341 297.7914078079565]


Now first of all this is hard to read back, especially if we want to loop over multiple regions

`cdutil` gives us the capability to create custom regions/domain.

In [2]:
import cdutil
elnino34 = cdutil.region.domain(latitude=(-5,5), longitude=(-170,-120))
tas_domain = ipsl_tas_file('tas', elnino34, time=("2000", "2010", "con")) # 10 years over ninon 34
el_nino34_domain = genutil.averager(tas_domain, axis='xy')
print(el_nino34_domain[:12])

[299.0817268861442 299.31519030799393 299.93105774638633 300.5384431714988
 300.5266711592158 299.58498040529855 299.1447297548061 298.748947382054
 298.27998928130893 297.90445208474205 297.4694662351559
 297.57607216636256]


Did you notice? These are different values?

The reason is that in addition to conveniently select your domain, the `domain` function resets the bounds on the selected slab to match exactly what you asked for. As a result `genutil.averager` will weights appropriately the cell going beyond your desired area.

Point:

In [3]:
def show_domain(data):
    blon = data.getLongitude().getBounds()
    blat = data.getLatitude().getBounds()
    print("\tLongitudes:", blon[0][0], blon[-1,1])
    print("\tLatitudes:", blat[0][0], blat[-1,1])

print("Regular selection domain:")
show_domain(tas)
print("cdutil selector domain:")
show_domain(tas_domain)

Regular selection domain:
	Longitudes: -171.25 -121.25
	Latitudes: -5.704225540161133 5.704225540161133
cdutil selector domain:
	Longitudes: -170.0 -120.0
	Latitudes: -5.0 5.0


In [4]:
print("Percentage Difference:", ((el_nino34_domain-el_nino34)/el_nino34_domain*100.)[:12])

Percentage Difference: [-0.04682863546914159 -0.034196005982305606 -0.02064204234148797
 -0.016851833627077296 -0.03145937686396504 -0.06156711429459702
 -0.06087850629238709 -0.06444440186421431 -0.0728994963603279
 -0.0774788038796271 -0.08006907024531786 -0.07236322464581055]


the `cdutil.region.domain` works for any dimension not just lat/lon.

Predefined region are: 'AAZ', 'AZ', 'AntarcticZone', 'ArcticZone', 'DomainComponent', 'NH', 'NPZ', 'NorthernHemisphere', 'SH', 'SPZ', 'SelectorComponent', 'SouthernHemisphere'

In [5]:
for region in ['AAZ', 'AZ', 'AntarcticZone', 'ArcticZone', 'NH', 'NPZ', 'NorthernHemisphere', 'SH', 'SPZ', 'SouthernHemisphere']:
    print("Region:", region, getattr(cdutil.region, region).components()[0].kargs)

Region: AAZ {'latitude': (None, -66.6)}
Region: AZ {'latitude': (66.6, None)}
Region: AntarcticZone {'latitude': (None, -66.6)}
Region: ArcticZone {'latitude': (66.6, None)}
Region: NH {'latitude': (0.0, None)}
Region: NPZ {'latitude': (66.6, None)}
Region: NorthernHemisphere {'latitude': (0.0, None)}
Region: SH {'latitude': (None, 0.0)}
Region: SPZ {'latitude': (None, -66.6)}
Region: SouthernHemisphere {'latitude': (None, 0.0)}


## Miscellaneous Region Functions

A more detailed notebook is available [here](https://cdat.llnl.gov/Jupyter-notebooks/cdat_utilities/Masking_Data_Land-Sea_masks/Masking_Data_Land-Sea_masks.html)

### Generating Land Sea Mask

Based on [this report](https://www-pcmdi.llnl.gov/publications/pdf/58.pdf)

In [6]:
tas_mask = cdutil.generateLandSeaMask(tas)
print(tas_mask.shape, tas_mask.max())  # No land in nino34

(9, 20) 0


### Generating Surface Type by Region

Again better described in the above-mentioned notebook

In [7]:
tas = ipsl_tas_file('tas', time=("2000", "2001", "con"))  # 1 year worth
tas_regions = cdutil.generateSurfaceTypeByRegionMask?

Signature:
cdutil.generateSurfaceTypeByRegionMask(
    mask,
    sftbyrgn=None,
    sftbyrgnmask=215,
    regions=[201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222],
    maximum_regions_per_cell=4,
    extend_up_to=3,
    verbose=True,
)
Docstring:
Maps a "regions" dataset onto a user provided land/sea mask or grid

:Example:

    .. doctest:: sftbyrgn_generateSurfaceTypeByRegionMask

        >>> mapped,found = generateSurfaceTypeByRegionMask(mask)

:param mask: User provided land/sea mask (100/0) or grid (the land/sea mask will be generated automagically)
        which will be mapped using the "sftbyrgn" internal dataset (will generate a land/sea mask for you)
:param sftbyrgn: Mask you wish to map onto your grid (if None uses internal "sftbyrgn" dataset (old ezget type))
:param sftbyrgnmask: Land/sea mask for sftbyrgn (or a number specifying value limits for sftbyrgn which indicates
        land/sea threshold (greater values 

Not mentioned in notebook is that you can provide your our "type region mask", with integers for each region

In [8]:
mask = cdutil.generateLandSeaMask(tas) * 100.
print(mask.shape)

(143, 144)


In [9]:
sft_by_rgn, failed = cdutil.generateSurfaceTypeByRegionMask(mask)

Regions: [201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222]
First pass
Done : |                                       | 0.00Done : ##                                      | 4.76Done : ####                                    | 9.52Done : ######                                  | 14.29Done : ########                                | 19.05Done : ##########                              | 23.81Done : ############                            | 28.57Done : ##############                          | 33.33Done : ################                        | 38.10Done : ##################                      | 42.86Done : ####################                    | 47.62Done : #####################                   | 52.38Done : #######################                 | 57.14Done : #########################               | 61.90Done : ###########################             | 66.67Done : #############################           | 71.43Done : 

In [10]:
import vcs
x = vcs.init()
x.plot(sft_by_rgn)
x.png("sft_by_rgn")

![](sft_by_rgn.png)